# DigitalGlobe

<div class="alert-info">

### Overview
    
* **teaching:** 15 minutes
* **exercises:** 0
* **questions:**
    * How can I find, anaylize, and visualize DigitalGlobe satellite imagery for an area of interest using Python?
    
</div>


This notebook will focus on accessing public datasets on AWS for a target area affected by Cyclone Kenneth (2019-04-25). Read more about this event and its impact at the [Humanitarian Open Street Map website](https://tasks.hotosm.org/project/5977). We will use a bounding box we will work with covers the island of Nagazidja, including the captial [city of Moroni](https://en.wikipedia.org/wiki/Moroni,_Comoros) - Union of the Comoros, a sovereign archipelago nation in the Indian Ocean. 

Sadly a lot of data out there is difficult to discover and manage. The [DigitalGlobe open data program](http://www.digitalglobe.com/ecosystem/open-data) is a tremendous resource, opening up traditionally costly high resolution optical imagery (50cm pixel postings!). However, we must manually find this website and download lists of image links without metadata or search functionality as we used with Landsat-8 and Sentinel-2.

In [ ]:
import pandas as pd
import geopandas as gpd

import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

import rasterio
import numpy as np
from ast import literal_eval

import shapely

from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

import matplotlib.pyplot as plt
%matplotlib inline

## Table of contents

1. [**DIY raster database**](#Simple-raster-database)
1. [**Visualize archive with holoviz**](#Visualize-archive-with-holoviz)
1. [**Cloud-optimized Geotiff virtual tiling**](#Cloud-optimized-Geotiff-virtual-tiling)

## Simple raster database

Since there is no STAC metadata for DigitalGlobe Open Data we need to create some sort of inventory to keep track of images. We'll use Geopandas for this, starting with a simple text file listing all the images on their website for Cyclone Kenneth https://www.digitalglobe.com/ecosystem/open-data/cyclone_kenneth 

In [ ]:
# Read the list of images (this was manually creating by downloading the lists from )
df = pd.read_csv('../assets/dg-open-cyclone-kenneth.txt', names=['url'])

In [ ]:
print(df.iloc[0:2].values)

In [ ]:
# Keep only .tif entries, not .ovr
df = df[df.url.str.endswith('tif')].reset_index(drop=True)

In [ ]:
# add some columns by parsing the url
df['datestr'] = df.url.str[60:70]
df['id'] = df.url.str[71:87]
df['frame'] = df.url.str[88:95]
df.head()

In [ ]:
df.info()

In [ ]:
url = df.iloc[0].values[0]
src = rasterio.open(url)
src.lnglat() # center point
#src.bounds # bounding box - would need to convert this to shapely polygon

In [ ]:
#Some files throw error when opening: CPLE_OpenFailedError: '/vsicurl/https://opendata.digitalglobe.com/cyclone-kenneth/pre-event/2018-09-27/1050010012588300/3323313.tif' not recognized as a supported file format.
def get_centroid(url):
    try:
        with rasterio.open(url) as src:
            #return Point(src.lnglat())
            return src.lnglat()
    except:
        print(f'no geometry for {url}')
        return np.NaN

In [ ]:
def get_bbox(url):
    try:
        with rasterio.open(url) as src:
            ll = (src.bounds.left, src.bounds.bottom)
            ul = (src.bounds.left, src.bounds.top)
            ur = (src.bounds.right, src.bounds.top)
            lr = (src.bounds.right, src.bounds.bottom)
            coords = (ll, ul, ur, lr, ll)
            #return Polygon(coords)
            return coords
    except:
        print(f'no geometry for {url}')
        return np.NaN

In [ ]:
def get_valid_footprint(url):
    # rasterio valid data mask
    print(f'rio shapes {url} --as-mask --bidx 1 --precision 5 --sampling 10')
    

In [ ]:
#get_valid_footprint(url)

In [ ]:
#%%time
# Running apply is slow, will take about 2s per url but only need to run once!
#df.url[:10].apply(get_centroid)

In [ ]:
#%%time
#df['centroid'] = df.url.apply(get_centroid)

In [ ]:
#%%time
#df['bbox'] = df.url.apply(get_bbox)

In [ ]:
#df.drop(columns='geometry', inplace=True)
#df.head()

In [ ]:
# Turn into geodataframe and visualize with pyviz
# Or just save to geojson and open on github / geojsonio / geojson jupyterhub extension
#df.info()

In [ ]:
#df.head()

In [ ]:
# Save this index for use later
#df.to_csv('dg-5877.csv', index=False)

In [ ]:
# Save this index for use later
df = pd.read_csv('../assets/dg-5877.csv')
df.head()

In [ ]:
shapely.geometry.Point(literal_eval(df.centroid.iloc[0]))
shapely.geometry.Polygon(literal_eval(df.bbox.iloc[0]))

In [ ]:
def to_shapely(string):
    return shapely.geometry.Polygon(literal_eval(string))

In [ ]:
geometries = df.bbox.apply(to_shapely)

In [ ]:
gf = gpd.GeoDataFrame(df, geometry=geometries, crs={'init': 'epsg:4326'})

In [ ]:
gf.head()

## Visualize archive with holoviz 

Plot an interactive heat map with raster tiles visualized.

In [ ]:
# Plot search AOI and frames on a map
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

# just keep id for hover tips
cols = gf.loc[:,('datestr','frame','geometry')]
footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Digital Globe Cyclone Kenneth Images')
#aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * labels

In [ ]:
frameid = '3323122'
subset = gf.query('frame == @frameid')
subset.head()

In [ ]:
# Pull some Digital Globe data based on shay's notebook
cogurl = subset.url.iloc[1]
with rasterio.open(cogurl) as src:
    print(src.profile)
    print(src.overviews(1))
    oviews = src.overviews(1)

In [ ]:
# Even higher resolution (4 meteres per pixel)
# We can load low resolution "overviews" from a cloud-optimized geotiff efficiently
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(cogurl) as src:
    oview = oviews[-1] # let's look at the smallest thumbnail
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(out_shape=(3, int(src.height // oview), int(src.width // oview)))
    
print('array type: ',type(thumbnail))
print(thumbnail)

In [ ]:
# NOTE: could plot
plt.imshow(thumbnail.transpose())
plt.title('{} Overview - RGB {}'.format(subset.frame.iloc[1], thumbnail.transpose().shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

##  Cloud-optimized Geotiff virtual tiling

This is super cool!!!! Use tiles.rdnt.io service to create full-resolution on the fly. Read more about this here https://medium.com/radiant-earth-insights/cog-map-and-tiles-rdnt-io-ad0745388a14

<div class="alert-warning">

#### Note 
    
As you zoom in resolution improves by as higher resolution tiles are created and fetched. Max zoom level for ipyleaflet maps is usually set to 18 
</div>


In [ ]:


service = 'https://tiles.rdnt.io/tiles/{z}/{x}/{y}'
rgbcog = subset.url.iloc[1]
center = subset.centroid.iloc[1].coords[0][::-1]

url = f'{service}?url={rgbcog}'
m = Map(center=center, zoom=12)

right_layer = TileLayer(url=url)
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

#m.add_layer(rectangle)

m